## Import Libraries

In [1]:
import pandas as pd
import numpy as np

### Define a Function to Print Describe and Info For a DF

In [2]:
# function to print describe and info for a dataframe
# input dataframe (df)
# returns nothing
def print_describe_and_info(df):
    print("Summary Statistics:")
    print(df.describe())
    
    print("\nInformation:")
    print(df.info())

## Data Cleansing
1. Filtered to New Jersey State
2. Removed Duplicates based on Address and Sold Date
3. Removed rows where required columns where null
4. Removed data that won't be useful like 0, 1 and > 6 beds, house_size < 800 sqft etc.
5. Added a new column called inf_price which is the componded price based on sold year. The inflation rate was assumed to be 2.5%
6. Remove Outliers

In [3]:
raw_df = pd.read_csv("master_data.csv", index_col='index')
print_describe_and_info(raw_df)

Summary Statistics:
            zip_code         price            bed           bath  \
count  742295.000000  7.424260e+05  624450.000000  628674.000000   
mean     5760.400575  7.250014e+05       3.422935       2.540585   
std      3747.934453  1.773573e+06       2.020496       1.994517   
min       601.000000  0.000000e+00       0.000000       0.500000   
25%      2536.000000  2.390000e+05       2.000000       2.000000   
50%      6016.000000  4.190000e+05       3.000000       2.000000   
75%      7930.000000  7.250000e+05       4.000000       3.000000   
max     99999.000000  8.750000e+08     123.000000     198.000000   

         house_size      sqft_lot  
count  6.359270e+05  5.817160e+05  
mean   2.172328e+03  8.213876e+05  
std    2.870146e+03  4.419060e+07  
min    1.000000e+02  0.000000e+00  
25%    1.243000e+03  6.098400e+03  
50%    1.727000e+03  1.481040e+04  
75%    2.499000e+03  5.880600e+04  
max    1.450112e+06  4.356000e+09  

Information:
<class 'pandas.core.frame.Dat

### Filter to New Jersey State, and Remove Duplicate Rows

In [4]:
new_jersey_df = raw_df[(raw_df["state"] == "New Jersey")]
# If there are two entries with same full_address and different sold_date, it means the house was sold twice at different times
new_jersey_df = new_jersey_df.drop_duplicates(subset=['full_address', 'sold_date'])
print_describe_and_info(new_jersey_df)
new_jersey_df.to_csv("realtor_data_dup.csv", index=False)

Summary Statistics:
           zip_code         price           bed          bath     house_size  \
count  41316.000000  4.131100e+04  36215.000000  36027.000000   36777.000000   
mean    7913.919014  6.339201e+05      3.469529      2.601132    2211.554477   
std      601.029641  1.254522e+06      1.432856      1.352659    2852.568158   
min     7001.000000  0.000000e+00      0.000000      0.500000     165.000000   
25%     7422.000000  2.750000e+05      3.000000      2.000000    1409.000000   
50%     8005.000000  4.300000e+05      3.000000      2.000000    1798.000000   
75%     8360.000000  6.890000e+05      4.000000      3.000000    2444.000000   
max    10303.000000  1.200000e+08     47.000000     39.000000  400149.000000   

           sqft_lot  
count  3.495100e+04  
mean   7.719125e+05  
std    3.993036e+07  
min    0.000000e+00  
25%    4.791600e+03  
50%    9.147600e+03  
75%    2.090880e+04  
max    4.356000e+09  

Information:
<class 'pandas.core.frame.DataFrame'>
Int64Inde

### Drop Rows Where Data Will Not be Useful
1. If these columns are not present, data is invalid - 'price', 'city', 'zip_code','sqft_lot'.
3. If bedroom is present, house_size cannot be null or 0, this means bogus data. However if both bedroom and house_size are not present, it means they are just selling the unconstructed land. Vice versa as well.
4. The company does not care about 0, 1 beds and > 6 beds
5. Remove house_size < 800 sqft. Not interested in it
6. Remove dates before 2003 since the prices may not be accurate.

In [5]:
# Dropna removes empty columns
#1 If these columns are not present, data is invalid - 'price', 'city', 'zip_code','sqft_lot'.
new_jersey_df = new_jersey_df.dropna(subset=['price', 'city', 'zip_code','sqft_lot'])
print_describe_and_info(new_jersey_df)

Summary Statistics:
           zip_code         price           bed          bath     house_size  \
count  34947.000000  3.494700e+04  30296.000000  30068.000000   30796.000000   
mean    7916.207028  6.466397e+05      3.638368      2.668185    2317.348097   
std      599.397597  1.328490e+06      1.389584      1.367713    3056.590838   
min     7001.000000  0.000000e+00      0.000000      0.500000     165.000000   
25%     7438.000000  2.850000e+05      3.000000      2.000000    1500.000000   
50%     8002.000000  4.440000e+05      3.000000      2.500000    1872.000000   
75%     8360.000000  6.900000e+05      4.000000      3.000000    2494.000000   
max     8904.000000  1.200000e+08     47.000000     39.000000  400149.000000   

           sqft_lot  
count  3.494700e+04  
mean   7.719843e+05  
std    3.993265e+07  
min    0.000000e+00  
25%    4.791600e+03  
50%    9.147600e+03  
75%    2.090880e+04  
max    4.356000e+09  

Information:
<class 'pandas.core.frame.DataFrame'>
Int64Inde

In [6]:
#2 Price, sqft_lot, house_size and bed cannot be 0.
zero_cond = (new_jersey_df['price'] != 0) & (new_jersey_df['sqft_lot'] != 0) & (new_jersey_df['house_size'] != 0) & (new_jersey_df['bed'] != 0)
new_jersey_df = new_jersey_df[zero_cond]
print_describe_and_info(new_jersey_df)

Summary Statistics:
           zip_code         price           bed          bath     house_size  \
count  34881.000000  3.488100e+04  30236.000000  30012.000000   30734.000000   
mean    7915.299848  6.471688e+05      3.641421      2.669349    2318.635192   
std      599.084299  1.329614e+06      1.388158      1.368222    3059.214950   
min     7001.000000  1.000000e+00      1.000000      0.500000     165.000000   
25%     7438.000000  2.850000e+05      3.000000      2.000000    1500.000000   
50%     8002.000000  4.450000e+05      3.000000      2.500000    1872.000000   
75%     8360.000000  6.900000e+05      4.000000      3.000000    2495.000000   
max     8904.000000  1.200000e+08     47.000000     39.000000  400149.000000   

           sqft_lot  
count  3.488100e+04  
mean   7.734107e+05  
std    3.997040e+07  
min    4.356000e+02  
25%    4.791600e+03  
50%    9.147600e+03  
75%    2.090880e+04  
max    4.356000e+09  

Information:
<class 'pandas.core.frame.DataFrame'>
Int64Inde

In [7]:
#4 Remove 0 or 1 or > 6 beds. Rows where bed is null will not be removed. null == val is false
bed_cond = ~((new_jersey_df['bed'] == 0) | (new_jersey_df['bed'] == 1) | (new_jersey_df['bed'] > 6))
new_jersey_df = new_jersey_df[bed_cond]
print_describe_and_info(new_jersey_df)

Summary Statistics:
           zip_code         price           bed          bath     house_size  \
count  33498.000000  3.349800e+04  28853.000000  28668.000000   29368.000000   
mean    7925.419130  6.249792e+05      3.544138      2.617064    2237.460706   
std      596.897133  1.282570e+06      1.041045      1.218729    2907.379533   
min     7001.000000  1.000000e+00      2.000000      1.000000     380.000000   
25%     7450.000000  2.890000e+05      3.000000      2.000000    1500.000000   
50%     8005.000000  4.400000e+05      3.000000      2.000000    1853.000000   
75%     8361.000000  6.824125e+05      4.000000      3.000000    2461.000000   
max     8904.000000  1.200000e+08      6.000000     15.000000  400149.000000   

           sqft_lot  
count  3.349800e+04  
mean   7.975969e+05  
std    4.078241e+07  
min    4.356000e+02  
25%    4.791600e+03  
50%    9.147600e+03  
75%    2.090880e+04  
max    4.356000e+09  

Information:
<class 'pandas.core.frame.DataFrame'>
Int64Inde

In [8]:
#5 Remove house size < 800
new_jersey_df = new_jersey_df[(new_jersey_df['house_size'].isna()) | (new_jersey_df['house_size'] >= 800)]
print_describe_and_info(new_jersey_df)

Summary Statistics:
           zip_code         price           bed          bath     house_size  \
count  33297.000000  3.329700e+04  28656.000000  28471.000000   29167.000000   
mean    7923.679461  6.272638e+05      3.553427      2.627305    2248.107210   
std      597.253563  1.286018e+06      1.037799      1.216087    2914.532361   
min     7001.000000  1.000000e+00      2.000000      1.000000     800.000000   
25%     7450.000000  2.899000e+05      3.000000      2.000000    1507.000000   
50%     8005.000000  4.420000e+05      3.000000      2.000000    1860.000000   
75%     8361.000000  6.850000e+05      4.000000      3.000000    2464.000000   
max     8904.000000  1.200000e+08      6.000000     15.000000  400149.000000   

           sqft_lot  
count  3.329700e+04  
mean   8.023370e+05  
std    4.090527e+07  
min    4.356000e+02  
25%    4.875000e+03  
50%    9.147600e+03  
75%    2.134400e+04  
max    4.356000e+09  

Information:
<class 'pandas.core.frame.DataFrame'>
Int64Inde

In [9]:
#3 If bedroom is present, house_size cannot be null or 0
house_bed_cond = ~(new_jersey_df['house_size'].notna() & ((new_jersey_df['bed'].isna()) | (new_jersey_df['bed'] == 0)))
new_jersey_df = new_jersey_df[house_bed_cond]
print_describe_and_info(new_jersey_df)

# Vice Versa bed present, but house_size empty
house_bed_cond = ~(new_jersey_df['bed'].notna() & ((new_jersey_df['house_size'].isna()) | (new_jersey_df['house_size'] == 0)))
new_jersey_df = new_jersey_df[house_bed_cond]
print_describe_and_info(new_jersey_df)

Summary Statistics:
           zip_code         price           bed          bath     house_size  \
count  32786.000000  3.278600e+04  28656.000000  28458.000000   28656.000000   
mean    7925.624169  6.244551e+05      3.553427      2.627556    2228.843802   
std      595.759164  1.293234e+06      1.037799      1.216034    2930.877637   
min     7001.000000  1.000000e+00      2.000000      1.000000     800.000000   
25%     7452.000000  2.899000e+05      3.000000      2.000000    1502.000000   
50%     8005.000000  4.399000e+05      3.000000      2.000000    1849.000000   
75%     8361.000000  6.799000e+05      4.000000      3.000000    2447.000000   
max     8904.000000  1.200000e+08      6.000000     15.000000  400149.000000   

           sqft_lot  
count  3.278600e+04  
mean   8.143783e+05  
std    4.122271e+07  
min    4.356000e+02  
25%    4.950000e+03  
50%    9.147600e+03  
75%    2.106150e+04  
max    4.356000e+09  

Information:
<class 'pandas.core.frame.DataFrame'>
Int64Inde

In [10]:
#6. Remove dates before 2003 since the prices may not be accurate.
new_jersey_df['sold_date'] = pd.to_datetime(new_jersey_df['sold_date'])
new_jersey_df = new_jersey_df[(new_jersey_df['sold_date'].isna()) | (new_jersey_df['sold_date'] >= '2003-01-01')]
print_describe_and_info(new_jersey_df)

Summary Statistics:
           zip_code         price           bed          bath     house_size  \
count  28126.000000  2.812600e+04  24239.000000  24066.000000   24239.000000   
mean    7909.231316  6.260265e+05      3.548537      2.622621    2231.390074   
std      596.549464  1.369443e+06      1.041127      1.214218    3134.386268   
min     7001.000000  1.000000e+00      2.000000      1.000000     800.000000   
25%     7436.000000  2.850000e+05      3.000000      2.000000    1500.000000   
50%     8002.000000  4.350000e+05      3.000000      2.000000    1843.000000   
75%     8342.750000  6.750000e+05      4.000000      3.000000    2444.000000   
max     8904.000000  1.200000e+08      6.000000     15.000000  400149.000000   

           sqft_lot  
count  2.812600e+04  
mean   6.438211e+05  
std    3.237064e+07  
min    4.356000e+02  
25%    4.791600e+03  
50%    9.147600e+03  
75%    2.072925e+04  
max    4.186987e+09  

Information:
<class 'pandas.core.frame.DataFrame'>
Int64Inde

### Additional Columns
1. Calculate inflation adjusted price apply a 2.5% inflation adjustment from the year part if year is not empty
2. calculate price/sqft_lot 
3. Calculate price/house_size

In [11]:
# Compound interest 2.5 percent p(1+r/100)pow n because inflation was 2.5%
inflation_factor = (1 + 2.5 / 100) ** (2023 - new_jersey_df['sold_date'].dt.year)
# where 1 is to keep theinf adjusted price same as price if year is 2023.
new_jersey_df['inf_price'] = new_jersey_df['price'] * inflation_factor.where(new_jersey_df['sold_date'].notnull(), 1)
new_jersey_df['inf_price_per_sqft_lot'] = new_jersey_df['inf_price'] / new_jersey_df['sqft_lot'].where(new_jersey_df['sqft_lot'].notna())
new_jersey_df['inf_price_per_house_size'] = new_jersey_df['inf_price'] / new_jersey_df['house_size'].where(new_jersey_df['house_size'].notna())
print_describe_and_info(new_jersey_df)


Summary Statistics:
           zip_code         price           bed          bath     house_size  \
count  28126.000000  2.812600e+04  24239.000000  24066.000000   24239.000000   
mean    7909.231316  6.260265e+05      3.548537      2.622621    2231.390074   
std      596.549464  1.369443e+06      1.041127      1.214218    3134.386268   
min     7001.000000  1.000000e+00      2.000000      1.000000     800.000000   
25%     7436.000000  2.850000e+05      3.000000      2.000000    1500.000000   
50%     8002.000000  4.350000e+05      3.000000      2.000000    1843.000000   
75%     8342.750000  6.750000e+05      4.000000      3.000000    2444.000000   
max     8904.000000  1.200000e+08      6.000000     15.000000  400149.000000   

           sqft_lot     inf_price  inf_price_per_sqft_lot  \
count  2.812600e+04  2.812600e+04            28126.000000   
mean   6.438211e+05  6.990759e+05               90.766200   
std    3.237064e+07  1.424782e+06              147.160687   
min    4.356000

## Remove outliers
1. Define a function to remove outliers from a df based on a column
2. Select rows where house_size is empty and remove outliers based on inf_price_per_sqft_lot
3. Select rows where house_size is not empty and remove outliers based on inf_price_per_house_size

We don't care about the price as such, only when price for a given land or house is too expensive or too cheap, we filter the data


In [12]:
# fuction to remove outliers
# input dataframe(df) and column name(col_name) - 'inf_price_per_sqft_lot' or 'inf_price_per_house_size'
# returns a df to caller with outlier rows removed

# value is outlier if value is < low_percentile - 1.5*iqr or value is > low_percentile +1.5*iqr

def remove_outliers(df, col_name):
   # calculate 10th percentile
   low_val = df[col_name].quantile(0.05)
   # calculate 90th percentile
   high_val = df[col_name].quantile(0.95)
   # run the condition on the df value is not outlier if value is >= low_percentile - 1.5*iqr or value is <= low_percentile +1.5*iqr
   df = df[(df[col_name] >= low_val) & (df[col_name] <= high_val)]
   return df

In [13]:
# Call the function where house_size is empty(land only sales)
lot_only_df = new_jersey_df[new_jersey_df['house_size'].isna()]
lot_only_df = remove_outliers(lot_only_df, 'inf_price_per_sqft_lot')
# Call the function where house_size is not empty(constructed home sales)
home_df = new_jersey_df[new_jersey_df['house_size'].notna()]
home_df = remove_outliers(home_df, 'inf_price_per_house_size')

new_jersey_df = pd.concat([lot_only_df, home_df])
print_describe_and_info(new_jersey_df)

Summary Statistics:
           zip_code         price           bed          bath    house_size  \
count  25312.000000  2.531200e+04  21815.000000  21713.000000  21815.000000   
mean    7906.469382  5.421844e+05      3.499106      2.556188   2091.279578   
std      603.892778  8.387946e+05      1.008030      1.100825    945.246519   
min     7001.000000  1.000000e+03      2.000000      1.000000    800.000000   
25%     7424.000000  2.990000e+05      3.000000      2.000000   1498.000000   
50%     7960.000000  4.300000e+05      3.000000      2.000000   1824.000000   
75%     8360.000000  6.490000e+05      4.000000      3.000000   2406.000000   
max     8904.000000  7.500000e+07      6.000000     11.000000  16000.000000   

           sqft_lot     inf_price  inf_price_per_sqft_lot  \
count  2.531200e+04  2.531200e+04            25312.000000   
mean   5.084587e+05  6.038327e+05               79.270208   
std    2.964769e+07  8.639314e+05              108.237584   
min    4.356000e+02  1.0

### Save the CSV file

In [14]:
### Ignore rows where sqft is > 20000, we don't care about bigger houses or farmlands etc

In [15]:
new_jersey_df = new_jersey_df[(new_jersey_df['house_size'].isna()) | ((new_jersey_df['sqft_lot'] >= 500) & (new_jersey_df['sqft_lot'] <= 20000))]
print_describe_and_info(new_jersey_df)
# save the data to a csv file
new_jersey_df.reset_index()
new_jersey_df.to_csv("realtor_data_cleaned.csv", index=True, index_label='index')

Summary Statistics:
           zip_code         price           bed         bath    house_size  \
count  20503.000000  2.050300e+04  17006.000000  16916.00000  17006.000000   
mean    7878.498073  4.984205e+05      3.415794      2.41044   1955.439904   
std      615.285448  8.931315e+05      0.988761      0.97406    752.592467   
min     7001.000000  1.000000e+03      2.000000      1.00000    800.000000   
25%     7305.000000  2.775000e+05      3.000000      2.00000   1467.000000   
50%     7927.000000  4.000000e+05      3.000000      2.00000   1760.000000   
75%     8332.000000  5.900000e+05      4.000000      3.00000   2310.000000   
max     8904.000000  7.500000e+07      6.000000     10.00000  15000.000000   

           sqft_lot     inf_price  inf_price_per_sqft_lot  \
count  2.050300e+04  2.050300e+04            20503.000000   
mean   3.714464e+04  5.526485e+05               87.516561   
std    2.277426e+05  9.099898e+05               92.247732   
min    4.356000e+02  1.000000e+03

### Insights
1. Data that is scrapped has a lot of duplicates and invalid rows.
2. It is important to adjust price, otherwise all the analysis becomes messes up
3. Extreme outliers need to be removed. Tukey's method was not able to remove lower outliers since q1-1.5*iqr was negative.